In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [2]:
breed_df = pd.read_excel('dog_breed_list.xlsx', names=['Breed'])
breed_df.head()

,Breed
0,affenpinscher
1,american_staffordshire_terrier
2,australian_terrier
3,basset
4,bernese_mountain_dog


In [3]:
breed_list = list(breed.replace('_', ' ').title() for breed in breed_df['Breed'])
breed_df['Use_for_Search'] = breed_list

In [4]:
# need to double check "Scotch Terrier" and "Toy Terrier", and all the list below

breed_df['Use_for_Search'][0] = 'Affenpinshcher'
breed_df['Use_for_Search'][3] = 'Basset Hound'
breed_df['Use_for_Search'][10] = 'Brittany'
breed_df['Use_for_Search'][11] = 'Bullmastiff'
breed_df['Use_for_Search'][17] = 'German Short-Haired'
breed_df['Use_for_Search'][29] = 'Scottish Terrier'
breed_df['Use_for_Search'][31] = 'Shetland Shepherd'
breed_df['Use_for_Search'][34] = 'Toy Fox Terrier'

In [5]:
breed_df.head()

,Breed,Use_for_Search
0,affenpinscher,Affenpinshcher
1,american_staffordshire_terrier,American Staffordshire Terrier
2,australian_terrier,Australian Terrier
3,basset,Basset Hound
4,bernese_mountain_dog,Bernese Mountain Dog


In [6]:
def openChrome(driver, op_url):
    chrome_options = Options() # browser open setting
    chrome_options.add_argument("--incognito")
    chrome = webdriver.Chrome(chrome_options=chrome_options,executable_path=driver)
    chrome.get(op_url)
    
    return(chrome)

In [7]:
# the chromedriver exe location
driver_dir = 'chromedriver.exe'

# open Chrome window and connect to the link
driver = openChrome(driver=driver_dir, op_url='https://ngdc.cncb.ac.cn/idog/breed/getAllBreed.action')
wait = WebDriverWait(driver, 5)
wait.until(EC.element_to_be_clickable((By.ID, 'form-breed-name')))
time.sleep(2)

C:\Users\s4013\AppData\Local\Temp\ipykernel_23252\649466626.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  chrome = webdriver.Chrome(chrome_options=chrome_options,executable_path=driver)
C:\Users\s4013\AppData\Local\Temp\ipykernel_23252\649466626.py:4: DeprecationWarning: use options instead of chrome_options
  chrome = webdriver.Chrome(chrome_options=chrome_options,executable_path=driver)


In [8]:
breed_details = []
breed_dict = {}

In [9]:
for breed in breed_df['Use_for_Search']:

    try:

        # locate the search input box and enter the breed name
        driver.find_elements(By.ID, 'form-breed-name')[0].find_element(By.TAG_NAME, 'input').clear()
        driver.find_elements(By.ID, 'form-breed-name')[0].find_element(By.TAG_NAME, 'input').send_keys(breed)
        time.sleep(1)
    
        # click search button
        driver.find_elements(By.CLASS_NAME, 'sub_button')[0].find_element(By.TAG_NAME, 'button').click()
        time.sleep(1)

        # create the XPATH based on the breed name
        if breed == 'Brittany':
            path = '/html/body/div/div[2]/div/div/div/div/div[2]/div/div[2]/ul/li[2]/div/div[1]/div/h4/a'
        else:
            path = r'//a[contains(text(), "{}")]'.format(breed)

        # click on the link to the breed info page
        driver.find_element(By.XPATH, path).click()
        time.sleep(1)

        # doing the scraping with BS
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # select the general info box
        general_info = soup.find('div', {'class': 'col-xs-12'})

        breed_dict['Breed'] = breed

        # loop through p tag and gather information in the general info box
        # append the info back to data list
        for i in general_info.find_all('p'):
            content = i.text.strip().replace('\t', '').replace('\n', '')
            trimmed = re.split(r'\s{2,}', content)
            trimmed = list(filter(None, trimmed))

            split_str = 'Energy Level'

            for tr in trimmed:
                if split_str in tr:
                    a = tr.split(split_str)[0]
                    b = split_str + tr.split(split_str)[1]

                    split_a = a.split(":")
                    breed_dict[split_a[0]] = split_a[1].strip()

                    split_b = b.split(":")
                    breed_dict[split_b[0]] = split_b[1].strip()

                split_data = tr.split(":")
                breed_dict[split_data[0]] = split_data[1].strip()
                
        breed_details.append(breed_dict.copy())                  
    
        # breed_details.append(data)

        print(f"Scraping for {breed} has been completed.")

        # get back to the original 
        # driver.find_element(By.XPATH, "//a[text()='All breeds']").click()

    except:
        print(f"Double check {breed}")


Scraping for Affenpinshcher has been completed.
Scraping for American Staffordshire Terrier has been completed.
Scraping for Australian Terrier has been completed.
Scraping for Basset Hound has been completed.
Scraping for Bernese Mountain Dog has been completed.
Scraping for Bloodhound has been completed.
Scraping for Border Collie has been completed.
Scraping for Border Terrier has been completed.
Scraping for Boxer has been completed.
Scraping for Briard has been completed.
Scraping for Brittany has been completed.
Scraping for Bullmastiff has been completed.
Scraping for Cocker Spaniel has been completed.
Scraping for Doberman has been completed.
Scraping for English Foxhound has been completed.
Scraping for French Bulldog has been completed.
Scraping for German Shepherd has been completed.
Scraping for German Short-Haired has been completed.
Scraping for Irish Terrier has been completed.
Scraping for Irish Water Spaniel has been completed.
Scraping for Italian Greyhound has been c

In [10]:
breed_details

[{'Breed': 'Affenpinshcher',
  'iDog Breed Number': 'CB1',
  'Original': 'Germany, France',
  'Personality': 'Loyal, curious, famously funny; fearless out of all proportion to their sizeEnergy Level',
  'Energy Level': 'Somewhat Active; Brisk walks, making new friends, and plenty of play will satisfy exercise needs',
  'Good With Children': 'Better with Supervision',
  'Good with other Dogs': 'With Supervision',
  'Shedding': 'Seasonal',
  'Grooming': 'Weekly',
  'Trainability': 'Eager To Please',
  'Height': '22.9-29.2 cm',
  'Weight': '3.2-4.5 kg',
  'Life Expectancy': '12-15 years',
  'Barking Level': 'Barks When Necessary',
  'History': '1936The Affenpinscher was officially recognized by the AKC.',
  'Web Source Name': 'Affenpinscherfrom AKCOther Name',
  'Common Name': 'Affen,Affie,Monkey Dog'},
 {'Breed': 'American Staffordshire Terrier',
  'iDog Breed Number': 'CB11',
  'Original': 'United States',
  'Personality': 'Confident, Smart, Good-natured Good With Children',
  'Energy L